In [32]:
from pathlib import Path
import pandas as pd

PATH_NOIPS = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian\goldeneye\hardware_log_20251213_190633.csv")
PATH_IPS   = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian_model\ae\goldeneye\hardware_log_20251215_020802.csv")

RAM_TOTAL_MB = 8192


In [33]:
def extract_metrics(csv_path):
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df["cpu_used_pct"] = 100 - df["cpu_idle_pct"]
    df["ram_used_mb"]  = RAM_TOTAL_MB - df["ram_available_mb"]
    df["disk_io_kbs"]  = df["disk_read_kbs"] + df["disk_write_kbs"]
    df["net_io_kbs"]   = df["net_rx_kbs"] + df["net_tx_kbs"]

    return {
        "cpu_mean": df["cpu_used_pct"].mean(),
        "cpu_max": df["cpu_used_pct"].max(),
        "ram_mean": df["ram_used_mb"].mean(),
        "ram_max": df["ram_used_mb"].max(),
        "disk_mean": df["disk_io_kbs"].mean(),
        "disk_max": df["disk_io_kbs"].max(),
        "net_mean": df["net_io_kbs"].mean(),
        "net_max": df["net_io_kbs"].max(),
    }


In [34]:
def process_single_csv(csv_path, scenario_name, attack_name):
    metrics = extract_metrics(csv_path)
    metrics["scenario"] = scenario_name
    metrics["attack"] = attack_name
    return pd.DataFrame([metrics])


In [35]:
noips_df = process_single_csv(
    PATH_NOIPS,
    scenario_name="no_ips",
    attack_name="goldeneye"
)

ips_df = process_single_csv(
    PATH_IPS,
    scenario_name="ips",
    attack_name="goldeneye"
)

all_df = pd.concat([noips_df, ips_df], ignore_index=True)


In [36]:
print(all_df.columns)
print(all_df)


Index(['cpu_mean', 'cpu_max', 'ram_mean', 'ram_max', 'disk_mean', 'disk_max',
       'net_mean', 'net_max', 'scenario', 'attack'],
      dtype='object')
    cpu_mean  cpu_max     ram_mean  ram_max    disk_mean  disk_max  \
0   4.877222    26.04  3006.114028  3100.47   349.312500  12701.48   
1  29.670426    73.71  3245.820000  3291.18  1526.934468  17464.10   

     net_mean  net_max scenario     attack  
0  106.122639  1319.33   no_ips  goldeneye  
1  282.455957  2296.51      ips  goldeneye  


In [37]:
mean_cols = ["cpu_mean", "ram_mean", "disk_mean", "net_mean"]

table_mean = (
    all_df
    .set_index(["attack", "scenario"])[mean_cols]
    .round(2)
)

print(table_mean)


                    cpu_mean  ram_mean  disk_mean  net_mean
attack    scenario                                         
goldeneye no_ips        4.88   3006.11     349.31    106.12
          ips          29.67   3245.82    1526.93    282.46


In [38]:
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

print(table_compare)


                    cpu_mean  cpu_max  ram_mean  ram_max  disk_mean  disk_max  \
attack    scenario                                                              
goldeneye ips          29.67    73.71   3245.82  3291.18    1526.93  17464.10   
          no_ips        4.88    26.04   3006.11  3100.47     349.31  12701.48   

                    net_mean  net_max  
attack    scenario                     
goldeneye ips         282.46  2296.51  
          no_ips      106.12  1319.33  


In [39]:
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).to_frame(name="delta")

print(delta_abs.round(2))


                 delta
cpu_mean     24.793203
ram_mean    239.705972
disk_mean  1177.621968
net_mean    176.333319


In [40]:
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).to_frame(name="delta_percent")

print(delta_pct.round(2))


          delta_percent
cpu_mean     508.346804
ram_mean       7.973948
disk_mean    337.125631
net_mean     166.159945


In [42]:
import pandas as pd

# ===============================
# 1) Tabel perbandingan utama
# ===============================
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

table_compare_out = table_compare.reset_index()
table_compare_out["section"] = "table_compare"


# ===============================
# 2) Delta absolut (IPS - No IPS)
# ===============================
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).round(2).to_frame(name="value")

delta_abs_out = delta_abs.reset_index().rename(columns={"index": "metric"})
delta_abs_out["section"] = "delta_absolute"


# ===============================
# 3) Delta persentase (%)
# ===============================
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).round(2).to_frame(name="value")

delta_pct_out = delta_pct.reset_index().rename(columns={"index": "metric"})
delta_pct_out["section"] = "delta_percent"


# ===============================
# 4) Gabungkan semua
# ===============================
final_csv = pd.concat(
    [table_compare_out, delta_abs_out, delta_pct_out],
    ignore_index=True
)

# ===============================
# 5) Simpan ke 1 CSV
# ===============================
final_csv.to_csv("comparison_goldeneye.csv", index=False)

print("[INFO] comparison_goldeneye.csv saved")


[INFO] comparison_goldeneye.csv saved
